In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO
from tensorflow.keras.models import load_model
classifier = load_model('model/tl_classifier.h5')
from keras.preprocessing import image

In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def predict_image(img):
    test_image=cv2.resize(img, (64, 64),
                       interpolation=cv2.INTER_CUBIC)
    test_image = np.array(test_image, dtype='float32')
    test_image=np.expand_dims(test_image,axis=0)
    result=classifier.predict(test_image)
    return result

In [4]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [5]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
    
        if cl == 9:
            crop_area = image[left:bottom,top:right]
            result=predict_image(crop_area)
            if result[0][0]==1:
                cv2.rectangle(image, (top, left), (right, bottom), (0, 0, 255), 2)
                cv2.putText(image, 'Red - Stop',(top, left - 6),cv2.FONT_HERSHEY_SIMPLEX,0.8, (0, 0, 255), 2,cv2.LINE_AA)
            elif result[0][0]==0:
                cv2.rectangle(image, (top, left), (right, bottom), (0, 255, 0), 2)
                cv2.putText(image, 'Green - Proceed',
                        (top, left - 6),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0, 255, 0), 2,
                        cv2.LINE_AA)
        else:
            continue

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [6]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [7]:

def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while camera.isOpened():
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(30) & 0xff == ord('q'):
                break

    vout.release()
    camera.release()
    cv2.destroyAllWindows()

In [8]:

# if __name__ == '__main__':
#     yolo = YOLO(0.6, 0.5)
#     file = 'data/coco_classes.txt'
#     all_classes = get_classes(file)


yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

In [9]:
# detect images in test floder.
for (root, dirs, files) in os.walk('images/test'):
    if files:
        for f in files:
            print(f)
            path = os.path.join(root, f)
            image = cv2.imread(path)
            image = detect_image(image, yolo, all_classes)
            cv2.imwrite('images/res/' + f, image)

dog.jpg
time: 24.57s

giraffe.jpg
time: 2.34s

t1.jpg
time: 2.12s
class: traffic light, score: 1.00
box coordinate x,y,w,h: [118.63767475  23.78337979 185.92112795 374.70880079]
class: traffic light, score: 0.76
box coordinate x,y,w,h: [420.76882267 142.55504751  73.44887322 113.8915897 ]

t2.jpg
time: 2.46s
class: traffic light, score: 0.98
box coordinate x,y,w,h: [175.20989227  40.70384018  54.58356714 120.3732653 ]

t3.jpg
time: 2.32s
t4.JPG
time: 2.38s

t5.jpg
time: 2.17s

tl.jpg
time: 2.10s



In [10]:
video = 'T2.mp4'
detect_video(video, yolo, all_classes)

time: 2.34s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 557.66942739  120.02800941  295.67233711 1378.91917155]

time: 2.20s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 558.75108719  123.82089615  292.99563468 1368.91639709]

time: 2.03s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 558.90764236  133.53435516  291.53478563 1346.18693865]

time: 2.09s
class: traffic light, score: 0.92
box coordinate x,y,w,h: [ 558.20134163  125.77208519  293.70310217 1358.52590891]

time: 2.11s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 558.47286701  116.54992819  291.95679903 1383.46737275]

time: 2.11s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 558.57985497  112.6848793   294.27777618 1385.98331782]

time: 2.00s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 556.34533882  120.73978901  297.45890558 1372.52325682]

time: 2.14s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 559.06361818  118.5642886

time: 2.01s
class: traffic light, score: 0.67
box coordinate x,y,w,h: [ 559.00330067   78.35810423  285.57670087 1410.00718887]

time: 2.05s
class: traffic light, score: 0.61
box coordinate x,y,w,h: [ 558.61178398   77.7513957   286.63144469 1409.8733029 ]

time: 2.08s
time: 2.22s
time: 2.05s
time: 1.92s
time: 2.02s
time: 2.05s
time: 2.06s
class: traffic light, score: 0.68
box coordinate x,y,w,h: [ 556.10966921   68.51084232  286.50671393 1418.6531287 ]

time: 2.12s
class: traffic light, score: 0.63
box coordinate x,y,w,h: [ 554.32409048   64.14175987  289.10324782 1430.18834738]

time: 2.14s
class: traffic light, score: 0.71
box coordinate x,y,w,h: [ 554.87235546   64.09054756  287.88148016 1427.58470645]

time: 1.96s
class: traffic light, score: 0.66
box coordinate x,y,w,h: [ 555.54878712   67.50149488  287.44800031 1427.03107871]

time: 1.90s
time: 2.04s
time: 1.97s
time: 2.22s
time: 2.13s
time: 1.99s
time: 2.11s
time: 2.04s
time: 2.00s
time: 2.00s
time: 2.20s
time: 2.06s
time: 2.14

time: 1.91s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 549.00507689  127.72411823  310.04092544 1284.58032791]

time: 1.93s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 547.19632387  128.71307373  315.56857198 1276.92867389]

time: 1.98s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.37136459  136.77450657  309.36795384 1264.86673539]

time: 2.02s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.28407478  131.51142597  306.39824152 1273.85779381]

time: 2.19s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.43621778  132.54007816  305.46267629 1270.71250842]

time: 2.02s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.35137415  137.09571362  304.71899897 1266.06179567]

time: 1.94s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.52714729  126.262393    300.43465286 1284.04674493]

time: 1.94s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 552.65824556  125.3588533